In [29]:
import os
import sys
from pathlib import Path

In [30]:
project_root = Path(__file__).parent.parent if "__file__" in locals() else Path().resolve().parent
sys.path.append(str(project_root))

In [31]:
from dataclasses import dataclass
from src.Briefly.utils.common import read_yaml, create_dirs
from src.Briefly.constants import *

import urllib.request as request
import zipfile 
from src.Briefly.logging import logger

In [32]:
CONFIG_PATH = project_root / CONFIG_PATH
PARAMS_PATH = project_root / PARAMS_PATH

Configurations:

In [33]:
@dataclass
class DataIngestionConfig:
    root_dir: Path
    source_url: Path
    local_data_file: Path
    unzip_dir: Path

In [34]:
class ConfigManager: 
    def __init__(self, config_path: Path = CONFIG_PATH, params_path: Path = PARAMS_PATH):
        self.config = read_yaml(config_path) 
        self.params = read_yaml(params_path)

        create_dirs([self.config.artifacts_root])
    def get_dataingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_dirs([config.root_dir,])
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_url=config.source_url,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        return data_ingestion_config

Components:

In [35]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url= self.config.source_url,
                filename= self.config.local_data_file
            )
            logger.info(f"Downloaded file from {self.config.source_url} to {filename}")
        else:
            logger.info(f"File already exists at {self.config.local_data_file}")
    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)

        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [36]:
config_manager = ConfigManager()
data_ingestion_config = config_manager.get_dataingestion_config()

data_ingestion = DataIngestion(data_ingestion_config)
data_ingestion.download_file()
data_ingestion.extract_zip_file()

[2025-01-07 14:15:00,877: INFO: common: YaML File: C:\Users\ABHINAV\Desktop\Programming\Projects\Briefly\config\config.yaml loaded successfully]
[2025-01-07 14:15:00,881: INFO: common: YaML File: C:\Users\ABHINAV\Desktop\Programming\Projects\Briefly\params.yaml loaded successfully]
[2025-01-07 14:15:00,882: INFO: common: Creating directory at: artifacts]
[2025-01-07 14:15:00,883: INFO: common: Creating directory at: artifacts/data_ingestion]
[2025-01-07 14:15:15,169: INFO: 3922641905: Downloaded file from https://drive.google.com/uc?export=download&id=1nwItANeU32CAD7w56tWIpjN709erAIyv to artifacts/data_ingestion/data.zip]
